# 2024-1 DSL 정규세션 과제

## CNN Assignment

- 작성자 : 윤형진
- 참고자료 : PyTorch Document
- 과제 제출 기한 : 02.18

---
- 이름 : 김홍서
- 기수 : 11기



In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [2]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# import가 안되어 직접 클래스 선언

import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

from PIL import Image

"""
MetaData.csv를 나누어 만든 train, val, test 데이터프레임에 담긴 이미지와 레이블을 반환해주는 CustomImageDataset을 만들어주세요
참고자료 : https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""

class CustomImageDataset(Dataset):

    def __init__(self, dataset_direc:str, df:pd.DataFrame, transform = None) -> None:

        # 3000개의 이미지를 담고있는 DataSet 디렉토리의 경로입니다.
        self.dataset_direc= "/content/drive/MyDrive/Colab Notebooks/DSL/DSL_assignments/CNN_HongSeoKim/DataSet/"

        self.df = df.reset_index(drop=True)

        self.to_tensor = ToTensor() # 파이토치에서는 모든 인풋을 torch.Tensor()로 바꿔줘야합니다.

        self.transform = transform

    # __init__() 함수에서 받은 데이터프레임에 포함된 전체 데이터포인트의 수를 반환하도록 해주세요
    def __len__(self) -> int :
        return len(self.df)


    def __getitem__(self, idx:int):
        # __init__() 함수에서 받은 데이터프레임의 각 행에서 받은 FileName, StrLabel, self.dataset_direc, Label을 이용해 이미지 데이터와 레이블을 반환해주세요

        path = path = os.path.join(self.dataset_direc, self.df.iloc[idx,1], self.df.iloc[idx,0])
        image = Image.open(path)
        label = self.df.iloc[idx,2]

        image = self.to_tensor(image)

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
import pandas as pd
import numpy as np

import yaml
import os

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import tqdm

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

"""
[Batch, Channel, 224, 224]의 input을 받았을 때,
최종적으로 sigmoid activation function에 들어갈 값을 반환하는 ResBottleNeck, ResNet50 Class를 정의해주세요.
즉, 정의된 ResNet50의 output 값은 확률이 아니라 sigmoid에 들어가는 값이 됩니다.
"""

def conv3x3(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels=in_channels,
                    out_channels=out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)

def conv1x1(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels=in_channels,
                    out_channels=out_channels, kernel_size=1,
                    stride=stride, padding=0, bias=False)


class ResBottleNeck(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1, donwsample=None) -> None:
        super(ResBottleNeck, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        # make the feature map size as half
        self.conv1 = conv1x1(in_channels, mid_channels, stride)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = conv3x3(mid_channels, mid_channels, stride=2)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.conv3 = conv1x1(mid_channels, out_channels, stride)

        self.donwsample = donwsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)

        if self.donwsample is not None:
            identity = self.donwsample(x)

        out += identity
        out = self.relu(out)
        return out


class ResNet50(nn.Module):
    def __init__(self) -> None:
        super(ResNet50,self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7,
                          stride=2, padding=3, bias=True),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace = True),
                        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                    )

        self.conv21 = ResBottleNeck(in_channels = 64, mid_channels=64, out_channels=256,
                                  stride=1, donwsample=conv1x1(in_channels=64, out_channels=256))
        self.conv22 = ResBottleNeck(in_channels = 256, mid_channels=64, out_channels=256,
                            stride=1, donwsample=None)
        self.conv23 = ResBottleNeck(in_channels = 256, mid_channels=64, out_channels=256,
                            stride=1, donwsample=None)

        self.conv31 = ResBottleNeck(in_channels = 256, mid_channels=128, out_channels=512,
                            stride=1, donwsample=conv1x1(in_channels=256, out_channels=512, stride = 2))
        self.conv32 = ResBottleNeck(in_channels = 512, mid_channels=128, out_channels=512,
                            stride=1, donwsample=None)
        self.conv33 = ResBottleNeck(in_channels = 512, mid_channels=128, out_channels=512,
                            stride=1, donwsample=None)
        self.conv34 = ResBottleNeck(in_channels = 512, mid_channels=128, out_channels=512,
                            stride=1, donwsample=None)

        self.conv41 = ResBottleNeck(in_channels = 512, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=conv1x1(in_channels=512, out_channels=1024, stride = 2))
        self.conv42 = ResBottleNeck(in_channels = 1024, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=None)
        self.conv43 = ResBottleNeck(in_channels = 1024, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=None)
        self.conv44 = ResBottleNeck(in_channels = 1024, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=None)
        self.conv45 = ResBottleNeck(in_channels = 1024, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=None)
        self.conv46 = ResBottleNeck(in_channels = 1024, mid_channels=256, out_channels=1024,
                            stride=1, donwsample=None)

        self.conv51 = ResBottleNeck(in_channels = 1024, mid_channels=512, out_channels=2048,
                            stride=1, donwsample=conv1x1(in_channels=1024, out_channels=2048, stride = 2))
        self.conv52 = ResBottleNeck(in_channels = 2048, mid_channels=512, out_channels=2048,
                            stride=1, donwsample=None)
        self.conv53 = ResBottleNeck(in_channels = 2048, mid_channels=512, out_channels=2048,
                            stride=1, donwsample=None)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, 1)

    def forward(self,x):

        #########################
        #  Froward Propagation  #
        #########################
        out = self.conv1(x)
        out = self.conv21(out)
        out = self.conv22(out)
        out = self.conv23(out)

        out = self.conv31(out)
        out = self.conv32(out)
        out = self.conv33(out)
        out = self.conv34(out)

        out = self.conv41(out)
        out = self.conv42(out)
        out = self.conv43(out)
        out = self.conv44(out)
        out = self.conv45(out)
        out = self.conv46(out)

        out = self.conv51(out)
        out = self.conv52(out)
        out = self.conv53(out)

        out = self.avgpool(out).squeeze(dim = -1).squeeze(dim = -1)
        out = self.fc(out).squeeze(dim = -1)

        return out

In [7]:
# config.yaml의 위치를 지정해주세요
config_path = "/content/drive/MyDrive/Colab Notebooks/DSL/DSL_assignments/CNN_HongSeoKim/scripts/config.yaml"

# 1. 하이퍼파라미터 및 기타 주요 변수 설정

config.yaml에 지정해줬던 디렉토리, 모델 하이퍼파라미터들을 가져옵시다

In [8]:
# config.yaml에서 디렉토리, 모델 하이퍼파라미터를 각각 받아옵니다
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

dataset_direc, meta_direc, weight_direc \
    = config["Direc"]["DatasetDirec"], \
      config["Direc"]["MetaDataDirec"], \
      config["Direc"]["WeightDirec"]

epoch, batch_size, lr \
    = config["HyperParameters"]["Epoch"], \
      config["HyperParameters"]["BatchSize"], \
      float(config["HyperParameters"]["LearningRate"]) # config.yaml에서 이미 float 자료형으로 학습률을 정의하신 분은 float()으로 자료형 변환을 하지 않으셔도 됩니다.

In [ ]:
import wandb # wandb.ai으로의 실시간 연동을 위한 라이브러리 임포트
from signal import signal, SIGPIPE, SIG_DFL
#Ignore SIG_PIPE and don't throw exceptions on it... (http://docs.python.org/library/signal.html)
signal(SIGPIPE,SIG_DFL)

# wandb login
wandb.init(
    project="DSL_ResNet", entity='hongseo-kim',
    name='hongseo-kim-cnn_no_augmentation',
    config={
        "model_name": "hongseo-kim-cnn_no_augmentation",
        "epochs": epoch,
        "batch_size": batch_size,
        "lr": lr
    })

config = wandb.config

In [10]:
# 가속을 위해 device로 GPU로 설정해줍니다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 2. train, val, test DataLoader 초기화

In [11]:
# MetaData.csv를 로드해주고, train, val, test로 나눠줍니다
meta_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DSL/DSL_assignments/CNN_HongSeoKim/scripts/MetaData.csv")

# train : val : test = 6 : 2 : 2
train, val_test = train_test_split(meta_data, train_size=0.6, random_state=777)
val, test = train_test_split(val_test, train_size=0.5, random_state=777)

# CustomDataset.py에서 가져온 CustomImageDataset 클래스에 train, val, test 데이터프레임을 패스하여 데이터셋을 만들어주세요
train_dataset = CustomImageDataset(dataset_direc=dataset_direc, df=train, transform = None)
val_dataset   = CustomImageDataset(dataset_direc=dataset_direc, df=val, transform = None)
test_dataset  = CustomImageDataset(dataset_direc=dataset_direc, df=test, transform = None)


train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader   = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_loader  = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

# 3. 모델 초기화, 손실함수 및 optimizer 지정

In [ ]:
# model을 선언하고 GPU에 올려줍니다
model = ResNet50()
model.to(device)

In [14]:
# 손실함수를 정의해주세요
criterion = torch.nn.BCEWithLogitsLoss()
# optimizer를 정의해주세요
optimizer = optim.Adam(model.parameters(), lr=lr)

# 4. Training 시작

학습을 위한 모든 준비가 끝났습니다! ㄱㅂㅈㄱ

In [17]:
# Best Validation Loss를 양의 무한대로 초기화
best_loss = float("inf")


# Training, Validation 시작

for curr_epoch in tqdm.tqdm(range(epoch+1)): # iterate over all EPOCHS
  train_loss = 0
  correct_train = 0

  for img, label in (train_loader): # iterate over all BATCHES

    # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
    img, label = img.to(device), label.to(device)

    # Forward Propagation
    pred = model(img)

    # 손실함수 계산
    loss = criterion(pred, label.float()) # 구현상의 차이로 label 텐서에 조작을 가해 모양을 맞춰줘야하는 경우가 생길 수 있습니다 ( 힌트 : tensor.view() )

    # 누적된 기울기 초기화 및 역전파
    optimizer.zero_grad()
    loss.backward()

    # 파라미터 업데이트
    optimizer.step()

    # 해당 Batch에서의 loss를 train_loss 변수에 누적
    train_loss += loss.item()

    # Accuracy 계산
    pred_class = (torch.sigmoid(pred) > 0.5).float()
    correct_train += (pred_class == label).float().sum()

  # 모든 Batch를 순회한 이후 최종적인 train_loss, train_acc를 계산해줍니다
  train_loss = train_loss/len(train_loader)
  train_acc = correct_train /len(train_loader)

  # Validation 시작
  model.eval()

  with torch.no_grad():  # 학습 중이 아니므로 역전파를 위한 gradient 연산은 필요하지 않습니다
    val_loss = 0
    correct_val = 0

    for img, label in (val_loader): # iterate over all BATCHES

      ##################################
      # forward propagation, loss 구현  #
      ##################################

      val_loss += loss.item()

      # 모든 Batch를 순회한 이후 최종적인 val_loss, val_acc를 계산해줍니다
      img, label = img.to(device), label.to(device)
      pred = model(img)
      pred_class = (torch.sigmoid(pred) > 0.5).float()
      correct_val += (pred_class == label).float().sum()

    val_loss /= len(val_loader)
    val_acc = correct_val / len(val_loader)

  # wandb.ai에 보낼 log 데이터를 작성해줍니다.
  metrics = {
    "epoch": epoch,
    "train_loss": train_loss,
    "val_loss" : val_loss,
    "train_acc": train_acc.item(),
    "val_acc" : val_acc.item(),
    "learning_rate": lr
    }

  # 아래 코드를 통해 실시간으로 학습현황을 확인할 수 있도록 log 데이터를 전송해줍니다
  wandb.log(metrics)

  # 모델을 다시 학습모드로 설정합니다
  model.train()

  # 가장 작은 validation loss를 가졌을 때의 모델 파라미터를 저장해줍니다.
  if val_loss < best_loss:
    best_loss = val_loss
    torch.save(model.state_dict(), os.path.join(weight_direc, "best_model(aug).pt")) # .pt 확장자로 저장해주도록합니다

wandb.finish()

100%|██████████| 41/41 [26:27<00:00, 38.71s/it]
